# 1. Creating the Scheme in the Local Host

### Create MYSQL connector

In [25]:
import MySQLdb as mdb

In [26]:
# creating connection
con = mdb.connect(
                host = '127.0.0.1', user = 'root', passwd = 'root') #optional - db="schema_name"  

In [31]:
# setting a cursor
cursor = con.cursor()     # get the cursor

### DDL - Create Schema

In [34]:
cursor.execute("DROP SCHEMA IF EXISTS Yelp_Pittsburgh ;")
cursor.execute("CREATE SCHEMA IF NOT EXISTS Yelp_Pittsburgh ;")
cursor.execute("USE Yelp_Pittsburgh ;")
cur.execute('SET SQL_MODE=ANSI_QUOTES') #important for petl operation - use this kind of quotes

0L

### Create Tables

In [18]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS students (
      `ID` int(11) NOT NULL,
      `Name` varchar(45) DEFAULT NULL,
      `Gender` varchar(45) DEFAULT NULL,
      PRIMARY KEY (`ID`)
        ) ;   """)

0L

## DML - Insert Statement

In [19]:
cursor.execute("INSERT INTO students VALUES ('3', 'Or', 'Male'), ('5', 'Ben', 'Male'), ('7', 'Or', 'Female');")
cursor.execute("SELECT * FROM students ")
print cursor.fetchall()

((3L, 'Or', 'Male'), (5L, 'Ben', 'Male'), (7L, 'Or', 'Female'))


Check the table via Workbech - what the difference?


### Commiting changes

In [20]:
con.commit()

### Closing the connection

In [21]:
#closing the connection
cursor.close()
con.close()